In [119]:
from riotwatcher import LolWatcher, ApiError

key = open('C:/Users/ThinkPad/Desktop/league test code/api_key.txt','r')
key=key.read()

lol_watcher = LolWatcher(key)

#Region codes BR1 EUN1 EUW1 JP1 KR LA1 LA2 NA1 OC1 TR1 RU servers. Codes might change as RIOT loves to make our life difficult
region = 'euw1' 

playerInfo = lol_watcher.summoner.by_name(region, 'Thebausffs')

try:
    response = lol_watcher.summoner.by_name(region, 'this_is_probably_not_anyones_summoner_name')
except ApiError as err:
    if err.response.status_code == 429:
        print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
        print('this retry-after is handled by default by the RiotWatcher library')
        print('future requests wait until the retry-after time passes')
    elif err.response.status_code == 404:
        print('')
    else:
        raise

In [120]:
player_ranked_stats = lol_watcher.league.by_summoner(region, playerInfo['id'])
player_ranked_stats

[{'leagueId': '038ccfd7-e490-4deb-a8bc-3530da13ef05',
  'queueType': 'RANKED_TFT_DOUBLE_UP',
  'tier': 'GOLD',
  'rank': 'II',
  'summonerId': '4WuWK6EA6X2FPLxi-thRXZCgjp2GhM9Ea8mCQ1f_3ocqVtKl',
  'summonerName': 'Thebausffs',
  'leaguePoints': 55,
  'wins': 43,
  'losses': 38,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'leagueId': 'f6134c21-afc2-3c72-8b43-5259ec512b58',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'MASTER',
  'rank': 'I',
  'summonerId': '4WuWK6EA6X2FPLxi-thRXZCgjp2GhM9Ea8mCQ1f_3ocqVtKl',
  'summonerName': 'Thebausffs',
  'leaguePoints': 483,
  'wins': 505,
  'losses': 488,
  'veteran': True,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False}]

In [121]:
import pandas as pd
pd.set_option('display.max_columns',None)
df = pd.DataFrame(player_ranked_stats)
pd.DataFrame(player_ranked_stats)

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,038ccfd7-e490-4deb-a8bc-3530da13ef05,RANKED_TFT_DOUBLE_UP,GOLD,II,4WuWK6EA6X2FPLxi-thRXZCgjp2GhM9Ea8mCQ1f_3ocqVtKl,Thebausffs,55,43,38,False,False,False,False
1,f6134c21-afc2-3c72-8b43-5259ec512b58,RANKED_SOLO_5x5,MASTER,I,4WuWK6EA6X2FPLxi-thRXZCgjp2GhM9Ea8mCQ1f_3ocqVtKl,Thebausffs,483,505,488,True,False,False,False


In [122]:
df1 = pd.DataFrame(player_ranked_stats, index = ["Ranked TFT",'Ranked Solo'])
df1 = df1.T
df1.loc[["tier",'rank','wins','losses']]

,Ranked TFT,Ranked Solo
tier,GOLD,MASTER
rank,II,I
wins,43,505
losses,38,488


In [123]:
playerInfo['puuid'] 

'b3SzCtkUnQbDY1IACZnTi_AqQBSnB2NPn9k1pH2P2r-PZ72zPrJ8-2iAsTTXy4GgNNonlwAg5gBdlg'

In [124]:
matchId = lol_watcher.match.matchlist_by_puuid(region, playerInfo['puuid']) #Fetching match Id from MATCH-V5 api

In [125]:
matchId # last 20 match ID

['EUW1_5989011352',
 'EUW1_5988965996',
 'EUW1_5988921212',
 'EUW1_5988876672',
 'EUW1_5987985934',
 'EUW1_5987663700',
 'EUW1_5987558556',
 'EUW1_5987165636',
 'EUW1_5987161025',
 'EUW1_5987086653',
 'EUW1_5987031002',
 'EUW1_5986994895',
 'EUW1_5986960658',
 'EUW1_5986856104',
 'EUW1_5986821490',
 'EUW1_5985982633',
 'EUW1_5985807622',
 'EUW1_5985802928',
 'EUW1_5985737042',
 'EUW1_5985723053']

In [126]:
#lol_watcher.match.by_id(region,matchId[15]) #Which match you wanna pull the data from 
#too large to display

In [127]:
import json

#if you want to save json file
#matchData = lol_watcher.match.by_id(region,matchId[0])
#with open('matchData.json', 'w') as json_file:
#    json.dump(matchData, json_file) 


In [128]:
matchData = lol_watcher.match.by_id(region,matchId[15])
matchData['info'].keys()

dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])

In [129]:
matchInfo = matchData['info']
#matchInfo['participants'] #too large to display

In [130]:
pd.DataFrame(matchInfo['participants'])

,assists,baronKills,basicPings,bountyLevel,challenges,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,8,0,23,1,"{'12AssistStreakCount': 0, 'abilityUses': 157,...",13609,15,150,Gnar,0,8,3601,5979,3601,21718,3,3,0,0,True,False,False,False,False,False,False,10238,8633,TOP,0,1,0,3143,1028,1033,2055,3078,3047,3363,22,0,2,TOP,0,0,1,626,9678,2081,7308,8,1,0,1,0,0,1,0,"{'statPerks': {'defense': 5002, 'flex': 5008, ...",94227,8330,14044,4864,Y31YNKNlma968-_j7zPqMOFYin5iHRjQBEEvg4I5xuSt4X...,0,,,DUO,0,110,16,27,4,4,4,3,12,BqE764iNexEPPEBRR-GgbK8dwEsVgqPpojpbLsJohvZbhAcJ,84,A UniQ Minion,False,100,TOP,17,1535,103967,10473,0,22608,3542,0,175,278,86,1,0,61,61,1255,2,4,4,0,28,4,4,11,True
1,8,1,91,0,"{'12AssistStreakCount': 0, 'abilityUses': 260,...",12236,14,203,Kindred,0,2,1001,34559,1001,14512,7,2,1,2,True,False,False,False,False,False,False,12958,12500,JUNGLE,0,0,0,2421,3031,3006,6676,1029,6672,3364,21,3,14,JUNGLE,1567,6,2,519,22777,2835,10071,123,0,0,0,0,0,2,0,"{'statPerks': {'defense': 5002, 'flex': 5008, ...",104874,14879,17721,5316,QabwxZpH6WWGhygUA-jxw1C3jY5Quei140LW-73YPzSx0C...,0,,,NONE,0,158,47,50,5,4,4,14,11,wNjuuoQ5wGB4F3MfrzEmBHau8VMikWVcW12nK0jHLFEqeT...,53,Meroltan,False,100,JUNGLE,11,1535,148072,20308,0,30033,11658,1642,8,216,176,9,0,20421,2593,2241,0,1,4,0,18,2,3,4,True
2,10,0,97,8,"{'12AssistStreakCount': 0, 'abilityUses': 213,...",14096,15,711,Vex,0,2,1958,8871,1958,8607,1,1,2,0,True,False,False,False,False,False,False,12671,11075,MIDDLE,0,1,0,3041,6655,0,1058,3158,4645,3364,16,1,8,TOP,0,8,2,163,133705,18922,5954,12,0,0,1,0,0,3,0,"{'statPerks': {'defense': 5003, 'flex': 5008, ...",12735,1845,8025,3150,amoFs1SHyoO6YkzIX7aem2_1dfq_HUbt2WGU-nmqg12K-P...,0,,,DUO,0,109,35,52,17,4,4,4,14,EXMLYDU4v5JssISDLp9aUG46IYVgk74sNu-zFNT404CBfTtC,474,PUNOKO KEKWww,False,100,MIDDLE,29,1535,151350,21770,0,15320,1890,0,195,669,10,1,0,4910,1002,1340,0,2,4,0,24,1,4,10,True
3,10,0,11,9,"{'12AssistStreakCount': 0, 'abilityUses': 226,...",13711,15,21,MissFortune,0,10,14206,15631,14206,7284,1,5,1,0,True,False,False,False,False,False,False,13402,11275,BOTTOM,1,1,0,3142,3134,0,3042,6692,3009,3363,27,1,10,BOTTOM,836,9,2,218,26333,3278,5578,5,0,0,1,0,0,4,0,"{'statPerks': {'defense': 5002, 'flex': 5008, ...",130745,16930,8030,1625,YmbeW82JjcZ6YVoPfcvHz1bAshWzl4mW4Zw3BcbIsfewbF...,0,,,CARRY,0,88,66,62,10,4,7,3,4,qw4rcik3osiLLCp2O6vOWHvdmPryEi4aTfV8UXHk1u501J...,76,hola bebe q más,False,100,BOTTOM,13,1535,158514,20369,0,14158,1016,364,207,394,10,3,0,1434,160,548,4,7,4,0,25,6,

In [131]:
#find out how many columns and their names
df2 = pd.DataFrame(matchInfo['participants'])
cols = df2.columns.tolist() 
cols

['assists',
 'baronKills',
 'basicPings',
 'bountyLevel',
 'challenges',
 'champExperience',
 'champLevel',
 'championId',
 'championName',
 'championTransform',
 'consumablesPurchased',
 'damageDealtToBuildings',
 'damageDealtToObjectives',
 'damageDealtToTurrets',
 'damageSelfMitigated',
 'deaths',
 'detectorWardsPlaced',
 'doubleKills',
 'dragonKills',
 'eligibleForProgression',
 'firstBloodAssist',
 'firstBloodKill',
 'firstTowerAssist',
 'firstTowerKill',
 'gameEndedInEarlySurrender',
 'gameEndedInSurrender',
 'goldEarned',
 'goldSpent',
 'individualPosition',
 'inhibitorKills',
 'inhibitorTakedowns',
 'inhibitorsLost',
 'item0',
 'item1',
 'item2',
 'item3',
 'item4',
 'item5',
 'item6',
 'itemsPurchased',
 'killingSprees',
 'kills',
 'lane',
 'largestCriticalStrike',
 'largestKillingSpree',
 'largestMultiKill',
 'longestTimeSpentLiving',
 'magicDamageDealt',
 'magicDamageDealtToChampions',
 'magicDamageTaken',
 'neutralMinionsKilled',
 'nexusKills',
 'nexusLost',
 'nexusTakedown

In [132]:
#Selecting the columns i need 
df3 = df2[['summonerName','championName','championId', 'champLevel', 'individualPosition','goldEarned','kills','deaths','assists','largestKillingSpree','largestMultiKill', 'totalDamageDealtToChampions', 'totalDamageTaken','objectivesStolen','baronKills','win']]
df3

,summonerName,championName,championId,champLevel,individualPosition,goldEarned,kills,deaths,assists,largestKillingSpree,largestMultiKill,totalDamageDealtToChampions,totalDamageTaken,objectivesStolen,baronKills,win
0,A UniQ Minion,Gnar,150,15,TOP,10238,2,3,8,0,1,10473,22608,0,0,True
1,Meroltan,Kindred,203,14,JUNGLE,12958,14,7,8,6,2,20308,30033,0,1,True
2,PUNOKO KEKWww,Vex,711,15,MIDDLE,12671,8,1,10,8,2,21770,15320,0,0,True
3,hola bebe q más,MissFortune,21,15,BOTTOM,13402,10,1,10,9,2,20369,14158,0,0,True
4,EBoy Selena Fan,Soraka,16,12,UTILITY,7526,0,2,19,0,0,3359,7255,0,0,True
5,Thebausffs,Quinn,133,13,TOP,9342,2,9,1,0,1,19376,22055,0,0,False
6,Anduril,Malzahar,90,13,MIDDLE,8959,2,5,4,0,1,13321,18693,0,0,False
7,Shunichi,RekSai,421,12,JUNGLE,8822,4,9,4,2,1,10696,25746,0,0,False
8,Void Puppy,KogMaw,96,14,BOTTOM,12096,4,4,5,2,2,16722,13011,0,0,False
9,weakside misery,Karma,43,10,UTILITY,6006,2,7,1,0,1,9221,13844,0,0,False


In [133]:
#Save new processed data in csv and json format to use them in Tableau
df3.to_csv('ProcessedMatchInfo.csv', encoding='utf-8')
df4 = df3.to_json()
with open('ProcessedMatchInfo.json', 'w') as json_file:
    json.dump(df4, json_file)